In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

In [3]:
limpieza = '"{}"'.format(os.path.join(current_folder, '..', 'Limpieza.ipynb'))

In [4]:
%run $limpieza
df_eventos_up_to_01062018 = get_clean_df()

In [5]:
df_features.shape

(38829, 0)

In [6]:
df_features.head()

""
person
4886f805
ad93850f
0297fc1e
2d681dd8
cccea85e


# Features por device

In [7]:
events_per_device_count = df_eventos_up_to_01062018.groupby('person')['device_type'].value_counts()
events_per_device_count.head()

person    device_type
0008ed71  Computer        2
00091926  Computer       34
00091a7a  Smartphone      1
000ba417  Computer        6
000c79fe  Smartphone      1
Name: device_type, dtype: int64

In [8]:
events_per_device_count= events_per_device_count.unstack().fillna(0)
events_per_device_count.head(10)

device_type,Computer,Smartphone,Tablet
person,,,
0008ed71,2.0,0.0,0.0
00091926,34.0,0.0,0.0
00091a7a,0.0,1.0,0.0
000ba417,6.0,0.0,0.0
000c79fe,0.0,1.0,0.0
000e4d9e,13.0,0.0,0.0
000e619d,5.0,0.0,0.0
001001be,0.0,3.0,0.0
0010e89a,1.0,0.0,0.0


In [9]:
df_features = df_features.merge(events_per_device_count, how='left', on='person')

In [10]:
df_features[events_per_device_count.columns].fillna(0, inplace=True)

Chequeo que no haya habido claves que se hayan quedado fuera del merge:

In [11]:
assert(df_features.shape[0] == df['person'].unique().shape[0])

In [12]:
del df_features['Tablet']
df_features.head()

,Computer,Smartphone
person,,
4886f805,0.0,1.0
ad93850f,0.0,5.0
0297fc1e,0.0,95.0
2d681dd8,2.0,0.0
cccea85e,22.0,0.0


# Feature vive en Brasil

In [13]:
df_eventos_up_to_01062018['vive en Brasil'] = (df_eventos_up_to_01062018['country']=='Brazil').astype(int)

In [14]:
df_vive_Brasil = df_eventos_up_to_01062018.groupby('person')[['vive en Brasil']].max()

In [15]:
df_vive_Brasil.head()

,vive en Brasil
person,
0008ed71,1
00091926,1
00091a7a,1
000ba417,1
000c79fe,1


In [16]:
df_features = df_features.merge(df_vive_Brasil, how='left', on='person')

Chequeo que no haya habido claves que se hayan quedado fuera del merge:

In [17]:
assert(df_features.shape[0] == df['person'].unique().shape[0])

In [18]:
df_features.shape

(38829, 3)

In [19]:
df_features.head()

,Computer,Smartphone,vive en Brasil
person,,,
4886f805,0.0,1.0,1
ad93850f,0.0,5.0,1
0297fc1e,0.0,95.0,1
2d681dd8,2.0,0.0,1
cccea85e,22.0,0.0,1


# Feature compra por mes 

In [20]:
df_cant_conversiones = df_eventos_up_to_01062018.loc[df_eventos_up_to_01062018['event']=='conversion',['person','timestamp']]
df_cant_conversiones['conv']=1
df_cant_conversiones.head()

,person,timestamp,conv
195,49c19e32,2018-05-18 01:12:20,1
234,49c19e32,2018-05-18 01:05:23,1
724,39df97e0,2018-05-29 07:51:14,1
744,380c0e60,2018-05-29 07:13:30,1
1198,35ee0cc8,2018-05-29 19:31:46,1


In [21]:
df_cant_conversiones['timestamp']= pd.to_datetime(df_cant_conversiones['timestamp'])
df_cant_conversiones.head()

,person,timestamp,conv
195,49c19e32,2018-05-18 01:12:20,1
234,49c19e32,2018-05-18 01:05:23,1
724,39df97e0,2018-05-29 07:51:14,1
744,380c0e60,2018-05-29 07:13:30,1
1198,35ee0cc8,2018-05-29 19:31:46,1


## Enero

In [22]:
df_compras_enero = df_cant_conversiones.loc[(df_cant_conversiones['timestamp'].dt.month==1),:]
df_compras_enero = df_compras_enero.groupby('person').agg({'conv':'sum'})
df_compras_enero.head()

,conv
person,
0008ed71,0
00091926,0
00091a7a,0
000ba417,0
000c79fe,0


In [23]:
df_features = df_features.merge(df_compras_enero, how='inner', on='person')
df_features.head()

,Computer,Smartphone,vive en Brasil,conv
person,,,,
4886f805,0.0,1.0,1,0
ad93850f,0.0,5.0,1,0
0297fc1e,0.0,95.0,1,0
2d681dd8,2.0,0.0,1,0
cccea85e,22.0,0.0,1,0


Chequeo que no haya habido claves que se hayan quedado fuera del merge:

In [24]:
assert(df_features.shape[0] == df['person'].unique().shape[0])

In [25]:
df_features = df_features.rename(columns={'Computer':'Cant visitas con Computadoras','Smartphone':'Cant visitas con smartphone','conv':'compras Enero'})

In [26]:
df_features.head()

,Cant visitas con Computadoras,Cant visitas con smartphone,vive en Brasil,compras Enero
person,,,,
4886f805,0.0,1.0,1,0
ad93850f,0.0,5.0,1,0
0297fc1e,0.0,95.0,1,0
2d681dd8,2.0,0.0,1,0
cccea85e,22.0,0.0,1,0


## Febrero

In [27]:
df_compras_febrero = df_cant_conversiones.loc[(df_cant_conversiones['timestamp'].dt.month==2),:]
df_compras_febrero = df_compras_febrero.groupby('person').agg({'conv':'sum'})
df_compras_febrero.head()

,conv
person,
0008ed71,0
00091926,0
00091a7a,0
000ba417,0
000c79fe,0


In [28]:
df_features = df_features.merge(df_compras_febrero, how='inner', on='person')
df_features.head()

,Cant visitas con Computadoras,Cant visitas con smartphone,vive en Brasil,compras Enero,conv
person,,,,,
4886f805,0.0,1.0,1,0,0
ad93850f,0.0,5.0,1,0,0
0297fc1e,0.0,95.0,1,0,0
2d681dd8,2.0,0.0,1,0,0
cccea85e,22.0,0.0,1,0,0


Chequeo que no haya habido claves que se hayan quedado fuera del merge:

In [29]:
assert(df_features.shape[0] == df['person'].unique().shape[0])

In [30]:
df_features.rename(columns={'conv':'compras Febrero'}, inplace=True)

df_features.head(10)

,Cant visitas con Computadoras,Cant visitas con smartphone,vive en Brasil,compras Enero,compras Febrero
person,,,,,
4886f805,0.0,1.0,1,0,0
ad93850f,0.0,5.0,1,0,0
0297fc1e,0.0,95.0,1,0,0
2d681dd8,2.0,0.0,1,0,0
cccea85e,22.0,0.0,1,0,0
4c8a8b93,0.0,20.0,1,0,0
1b9f7cf6,0.0,3.0,1,0,0
29ebb414,0.0,4.0,1,0,0
de8fe91b,2.0,0.0,1,0,0


## Marzo

In [31]:
df_compras_marzo = df_cant_conversiones.loc[(df_cant_conversiones['timestamp'].dt.month==3),:]
df_compras_marzo = df_compras_marzo.groupby('person').agg({'conv':'sum'})
df_features = df_features.merge(df_compras_marzo, how='inner', on='person')
df_features.rename(columns={'conv':'compras Marzo'}, inplace=True)

df_features.head(10)

,Cant visitas con Computadoras,Cant visitas con smartphone,vive en Brasil,compras Enero,compras Febrero,compras Marzo
person,,,,,,
4886f805,0.0,1.0,1,0,0,0
ad93850f,0.0,5.0,1,0,0,0
0297fc1e,0.0,95.0,1,0,0,0
2d681dd8,2.0,0.0,1,0,0,0
cccea85e,22.0,0.0,1,0,0,0
4c8a8b93,0.0,20.0,1,0,0,0
1b9f7cf6,0.0,3.0,1,0,0,0
29ebb414,0.0,4.0,1,0,0,0
de8fe91b,2.0,0.0,1,0,0,0


In [32]:
assert(df_features.shape[0] == df['person'].unique().shape[0])

## Abril

In [33]:
df_compras_abril = df_cant_conversiones.loc[(df_cant_conversiones['timestamp'].dt.month==4),:]
df_compras_abril = df_compras_abril.groupby('person').agg({'conv':'sum'})
df_features = df_features.merge(df_compras_abril, how='inner', on='person')
df_features.rename(columns={'conv':'compras Abril'}, inplace=True)


In [34]:
assert(df_features.shape[0] == df['person'].unique().shape[0])

## Mayo

In [35]:
df_compras_mayo = df_cant_conversiones.loc[(df_cant_conversiones['timestamp'].dt.month==5),:]
df_compras_mayo = df_compras_mayo.groupby('person').agg({'conv':'sum'})
df_features = df_features.merge(df_compras_mayo, how='inner', on='person')
df_features.rename(columns={'conv':'compras Mayo'}, inplace=True)

In [36]:
assert(df_features.shape[0] == df['person'].unique().shape[0])

In [37]:
df_features.head()

,Cant visitas con Computadoras,Cant visitas con smartphone,vive en Brasil,compras Enero,compras Febrero,compras Marzo,compras Abril,compras Mayo
person,,,,,,,,
4886f805,0.0,1.0,1,0,0,0,0,0
ad93850f,0.0,5.0,1,0,0,0,0,0
0297fc1e,0.0,95.0,1,0,0,0,0,0
2d681dd8,2.0,0.0,1,0,0,0,0,0
cccea85e,22.0,0.0,1,0,0,0,0,0


# Cantidad de días desde la última compra.

In [38]:
df_eventos_up_to_01062018['timestamp']= pd.to_datetime(df_eventos_up_to_01062018['timestamp'])
df_ordenado_tiempo = df_eventos_up_to_01062018.sort_values('timestamp')

In [39]:
df_ordenado_tiempo.head(10)

,timestamp,event,person,url,sku,model,condition,storage_gb,color,skus,search_term,staticpage,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,screen_resolution_width,screen_resolution_height,vive en Brasil
1753202,2018-01-01 08:09:31,search engine hit,0f4e2a4b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1753201,2018-01-01 08:09:31,ad campaign hit,0f4e2a4b,/comprar/iphone/6-plus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2307205,2018-01-01 08:09:31,visited site,0f4e2a4b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Paid,New,São Paulo,Sao Paulo,Brazil,Computer,1366x768,Windows 8.1,Chrome 63.0,1366.0,768.0,1
1753200,2018-01-01 08:09:44,viewed product,0f4e2a4b,NaN,1397.0,iPhone 6 Plus,Excelente,64.0,Plateado,"2740,2830,2741,2692,6819,2702,2691,2703,6791,6...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1753210,2018-01-01 08:45:29,search engine hit,0f4e2a4b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1753207,2018-01-01 08:45:29,ad campaign hit,0f4e2a4b,/comprar/iphone/6-plus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2307206,2018-01-01 08:45:29,visited site,0f4e2a4b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Paid,Returning,São Paulo,Sao Paulo,Brazil,Computer,1366x768,Windows 8.1,Chrome 63.0,1366.0,768.0,1
1753203,2018-01-01 08:45:43,viewed product,0f4e2a4b,NaN,2843.0,iPhone 6 Plus,Bueno,64.0,Dorado,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1753204,2018-01-01 08:48:57,viewed product,0f4e2a4b,NaN,2841.0,iPhone 6 Plus,Bueno,64.0,Plateado,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1753213,2018-01-01 08:49:05,viewed product,0f4e2a4b,NaN,2843.0,iPhone 6 Plus,Bueno,64.0,Dorado,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [40]:
df_ordenado_tiempo = df_ordenado_tiempo.loc[df_ordenado_tiempo['event']=='conversion',['timestamp','person']]

In [41]:
df_ultima_compra = df_ordenado_tiempo.groupby('person').max()
df_ultima_compra.head()

,timestamp
person,
0008ed71,NaT
00091926,NaT
00091a7a,NaT
000ba417,2018-05-26 13:09:22
000c79fe,NaT


In [42]:
df_ultima_compra['timestamp'].fillna(pd.to_datetime('2017-12-31 15:00:00'),inplace=True)

In [43]:
def diff_dias(col):
    return (pd.to_datetime('2018-06-01 00:00:00').date()-col.date())

In [44]:
df_ultima_compra['dias ultima compra'] = df_ultima_compra['timestamp'].transform(diff_dias)

In [45]:
df_ultima_compra['dias ultima compra'] = df_ultima_compra['dias ultima compra'].dt.days

In [46]:
del df_ultima_compra['timestamp']
df_ultima_compra.head()

,dias ultima compra
person,
0008ed71,152
00091926,152
00091a7a,152
000ba417,6
000c79fe,152


In [47]:
df_features = df_features.merge(df_ultima_compra, how='left', on='person')
df_features.head()

,Cant visitas con Computadoras,Cant visitas con smartphone,vive en Brasil,compras Enero,compras Febrero,compras Marzo,compras Abril,compras Mayo,dias ultima compra
person,,,,,,,,,
4886f805,0.0,1.0,1,0,0,0,0,0,152
ad93850f,0.0,5.0,1,0,0,0,0,0,152
0297fc1e,0.0,95.0,1,0,0,0,0,0,152
2d681dd8,2.0,0.0,1,0,0,0,0,0,152
cccea85e,22.0,0.0,1,0,0,0,0,0,152


In [48]:
assert(df_features.shape[0] == df_eventos_up_to_01062018['person'].unique().shape[0])

# Cantidad de días desde el ùltimo checkout

In [49]:
df_eventos_up_to_01062018['timestamp']= pd.to_datetime(df_eventos_up_to_01062018['timestamp'])
df_ordenado_tiempo = df_eventos_up_to_01062018.sort_values('timestamp')

In [50]:
df_ordenado_tiempo.head(10)

,timestamp,event,person,url,sku,model,condition,storage_gb,color,skus,search_term,staticpage,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,screen_resolution_width,screen_resolution_height,vive en Brasil
1753202,2018-01-01 08:09:31,search engine hit,0f4e2a4b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1753201,2018-01-01 08:09:31,ad campaign hit,0f4e2a4b,/comprar/iphone/6-plus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2307205,2018-01-01 08:09:31,visited site,0f4e2a4b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Paid,New,São Paulo,Sao Paulo,Brazil,Computer,1366x768,Windows 8.1,Chrome 63.0,1366.0,768.0,1
1753200,2018-01-01 08:09:44,viewed product,0f4e2a4b,NaN,1397.0,iPhone 6 Plus,Excelente,64.0,Plateado,"2740,2830,2741,2692,6819,2702,2691,2703,6791,6...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1753210,2018-01-01 08:45:29,search engine hit,0f4e2a4b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1753207,2018-01-01 08:45:29,ad campaign hit,0f4e2a4b,/comprar/iphone/6-plus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2307206,2018-01-01 08:45:29,visited site,0f4e2a4b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Paid,Returning,São Paulo,Sao Paulo,Brazil,Computer,1366x768,Windows 8.1,Chrome 63.0,1366.0,768.0,1
1753203,2018-01-01 08:45:43,viewed product,0f4e2a4b,NaN,2843.0,iPhone 6 Plus,Bueno,64.0,Dorado,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1753204,2018-01-01 08:48:57,viewed product,0f4e2a4b,NaN,2841.0,iPhone 6 Plus,Bueno,64.0,Plateado,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1753213,2018-01-01 08:49:05,viewed product,0f4e2a4b,NaN,2843.0,iPhone 6 Plus,Bueno,64.0,Dorado,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [51]:
df_ordenado_tiempo = df_ordenado_tiempo.loc[df_ordenado_tiempo['event']=='checkout',['timestamp','person']]

In [52]:
df_ultimo_checkout = df_ordenado_tiempo.groupby('person').max()
df_ultimo_checkout.head()

,timestamp
person,
0008ed71,2018-05-17 16:28:37
00091926,2018-05-26 00:30:47
00091a7a,NaT
000ba417,2018-05-26 13:03:02
000c79fe,2018-05-29 00:29:51


In [53]:
df_ultimo_checkout['timestamp'].fillna(pd.to_datetime('2017-12-31 15:00:00'),inplace=True)

In [54]:
df_ultimo_checkout['dias ultimo checkout'] = df_ultimo_checkout['timestamp'].transform(diff_dias)

In [55]:
df_ultimo_checkout['dias ultimo checkout'] = df_ultimo_checkout['dias ultimo checkout'].dt.days

In [56]:
del df_ultimo_checkout['timestamp']
df_ultimo_checkout.head()

,dias ultimo checkout
person,
0008ed71,15
00091926,6
00091a7a,152
000ba417,6
000c79fe,3


In [57]:
df_features = df_features.merge(df_ultimo_checkout, how='left', on='person')
df_features.head()

,Cant visitas con Computadoras,Cant visitas con smartphone,vive en Brasil,compras Enero,compras Febrero,compras Marzo,compras Abril,compras Mayo,dias ultima compra,dias ultimo checkout
person,,,,,,,,,,
4886f805,0.0,1.0,1,0,0,0,0,0,152,14
ad93850f,0.0,5.0,1,0,0,0,0,0,152,18
0297fc1e,0.0,95.0,1,0,0,0,0,0,152,10
2d681dd8,2.0,0.0,1,0,0,0,0,0,152,5
cccea85e,22.0,0.0,1,0,0,0,0,0,152,21


In [58]:
assert(df_features.shape[0] == df_eventos_up_to_01062018['person'].unique().shape[0])

# Eventos según día 

In [59]:
df_eventos_up_to_01062018 = df_eventos_up_to_01062018.loc[:,['person','timestamp']]
df_eventos_up_to_01062018['evento'] = 1

In [60]:
df_eventos_up_to_01062018['timestamp'] = pd.to_datetime(df_eventos_up_to_01062018['timestamp'])

In [61]:
df_eventos_up_to_01062018['dia'] = pd.to_datetime(df_eventos_up_to_01062018['timestamp']).dt.weekday_name

In [62]:
df_eventos_up_to_01062018.head()

,person,timestamp,evento,dia
0,4886f805,2018-05-18 00:11:59,1,Friday
1,ad93850f,2018-05-18 00:11:27,1,Friday
2,0297fc1e,2018-05-18 00:11:16,1,Friday
3,2d681dd8,2018-05-18 00:11:14,1,Friday
4,cccea85e,2018-05-18 00:11:09,1,Friday


In [63]:
cant_eventos_x_dia_semana = df_eventos_up_to_01062018.groupby(['person','dia']).agg({'evento':'sum'})
cant_eventos_x_dia_semana.head()

evento
person   dia             
0008ed71 Thursday       6
00091926 Friday        86
         Monday        32
         Saturday      32
         Sunday       131

In [64]:

cant_eventos_x_dia_semana =cant_eventos_x_dia_semana.unstack()

In [65]:
cant_eventos_x_dia_semana = cant_eventos_x_dia_semana.fillna(0)
cant_eventos_x_dia_semana.head()

evento                                                  
dia      Friday Monday Saturday Sunday Thursday Tuesday Wednesday
person                                                           
0008ed71    0.0    0.0      0.0    0.0      6.0     0.0       0.0
00091926   86.0   32.0     32.0  131.0     84.0    43.0      40.0
00091a7a    0.0   10.0      0.0    0.0      0.0     0.0       0.0
000ba417    0.0    0.0     59.0    0.0    147.0     0.0       0.0
000c79fe    0.0    0.0      0.0    0.0      0.0    17.0       0.0

In [66]:
cant_eventos_x_dia_semana.columns = ['eventos Viernes','eventos Lunes','eventos Sabado','eventos Domingo','eventos Jueves','eventos Martes','eventos Miercoles']
cant_eventos_x_dia_semana.head()

,eventos Viernes,eventos Lunes,eventos Sabado,eventos Domingo,eventos Jueves,eventos Martes,eventos Miercoles
person,,,,,,,
0008ed71,0.0,0.0,0.0,0.0,6.0,0.0,0.0
00091926,86.0,32.0,32.0,131.0,84.0,43.0,40.0
00091a7a,0.0,10.0,0.0,0.0,0.0,0.0,0.0
000ba417,0.0,0.0,59.0,0.0,147.0,0.0,0.0
000c79fe,0.0,0.0,0.0,0.0,0.0,17.0,0.0


In [67]:
df_features = df_features.merge(cant_eventos_x_dia_semana, how='left', on='person')

In [68]:
assert(df_features.shape[0] == df_eventos_up_to_01062018['person'].unique().shape[0])

# Eventos por franja horaria

In [69]:
df_eventos_up_to_01062018['hora'] = pd.to_datetime(df_eventos_up_to_01062018['timestamp']).dt.hour

In [70]:
df_eventos_up_to_01062018['madrugada'] = np.where(((df_eventos_up_to_01062018['hora']<7) & (df_eventos_up_to_01062018['hora']>=0)), 1,0)
df_eventos_up_to_01062018['maniana'] = np.where(((df_eventos_up_to_01062018['hora']<13) & (df_eventos_up_to_01062018['hora']>=7)), 1,0)
df_eventos_up_to_01062018['tarde'] = np.where(((df_eventos_up_to_01062018['hora']<19) & (df_eventos_up_to_01062018['hora']>=13)), 1,0)
df_eventos_up_to_01062018['noche'] = np.where(((df_eventos_up_to_01062018['hora']<24) & (df_eventos_up_to_01062018['hora']>=19)), 1,0)
df_eventos_up_to_01062018.head()

,person,timestamp,evento,dia,hora,madrugada,maniana,tarde,noche
0,4886f805,2018-05-18 00:11:59,1,Friday,0,1,0,0,0
1,ad93850f,2018-05-18 00:11:27,1,Friday,0,1,0,0,0
2,0297fc1e,2018-05-18 00:11:16,1,Friday,0,1,0,0,0
3,2d681dd8,2018-05-18 00:11:14,1,Friday,0,1,0,0,0
4,cccea85e,2018-05-18 00:11:09,1,Friday,0,1,0,0,0


In [71]:
eventos_hora = df_eventos_up_to_01062018.groupby(['person']).agg({'madrugada':'sum','maniana':'sum','tarde':'sum','noche':'sum'})
eventos_hora.head()

,madrugada,maniana,tarde,noche
person,,,,
0008ed71,0,1,5,0
00091926,313,0,50,85
00091a7a,0,0,10,0
000ba417,0,100,106,0
000c79fe,17,0,0,0


In [72]:
eventos_hora.columns = ['eventos madrugada','eventos maniana','eventos tarde','eventos noche']

In [73]:
df_features = df_features.merge(eventos_hora, how='left', on='person')
assert(df_features.shape[0] == df_eventos_up_to_01062018['person'].unique().shape[0])

# Eventos segun dia y franja horaria

In [74]:
df_eventos_up_to_01062018.head()

,person,timestamp,evento,dia,hora,madrugada,maniana,tarde,noche
0,4886f805,2018-05-18 00:11:59,1,Friday,0,1,0,0,0
1,ad93850f,2018-05-18 00:11:27,1,Friday,0,1,0,0,0
2,0297fc1e,2018-05-18 00:11:16,1,Friday,0,1,0,0,0
3,2d681dd8,2018-05-18 00:11:14,1,Friday,0,1,0,0,0
4,cccea85e,2018-05-18 00:11:09,1,Friday,0,1,0,0,0


In [75]:
eventos_dia_hora = df_eventos_up_to_01062018.groupby(['person','dia']).agg({'madrugada':'sum','maniana':'sum','tarde':'sum','noche':'sum'})

In [76]:
eventos_dia_hora.columns = ['eventos madrugada','eventos maniana','eventos tarde','eventos noche']

In [77]:
eventos_dia_hora.head()

eventos madrugada  eventos maniana  eventos tarde  \
person   dia                                                           
0008ed71 Thursday                  0                1              5   
00091926 Friday                   86                0              0   
         Monday                   11                0              0   
         Saturday                 16                0              0   
         Sunday                   73                0             50   

                   eventos noche  
person   dia                      
0008ed71 Thursday              0  
00091926 Friday                0  
         Monday               21  
         Saturday             16  
         Sunday                8

In [78]:
eventos_dia_hora=eventos_dia_hora.unstack()

In [79]:
eventos_dia_hora =eventos_dia_hora.fillna(0)

In [80]:
eventos_dia_hora.columns = ['Evento Madrugada Viernes','Evento Madrugada Lunes','Evento Madrugada Sabado','Evento Madrugada Domingo','Evento Madrugada Jueves','Evento Madrugada Martes','Evento Madrugada Miercoles','Evento Maniana Viernes','Evento Maniana Lunes','Evento Maniana Sabado','Evento Maniana Domingo','Evento Maniana Jueves','Evento Maniana Martes','Evento Maniana Miercoles','Evento Tarde Viernes','Evento Tarde Lunes','Evento Tarde Sabado','Evento Tarde Domingo','Evento Tarde Jueves','Evento Tarde Martes','Evento Tarde Miercoles','Evento Noche Viernes','Evento Noche Lunes','Evento Noche Sabado','Evento Noche Domingo','Evento Noche Jueves','Evento Noche Martes','Evento Noche Miercoles']
                    

In [81]:
eventos_dia_hora.head()

,Evento Madrugada Viernes,Evento Madrugada Lunes,Evento Madrugada Sabado,Evento Madrugada Domingo,Evento Madrugada Jueves,Evento Madrugada Martes,Evento Madrugada Miercoles,Evento Maniana Viernes,Evento Maniana Lunes,Evento Maniana Sabado,Evento Maniana Domingo,Evento Maniana Jueves,...,Evento Tarde Sabado,Evento Tarde Domingo,Evento Tarde Jueves,Evento Tarde Martes,Evento Tarde Miercoles,Evento Noche Viernes,Evento Noche Lunes,Evento Noche Sabado,Evento Noche Domingo,Evento Noche Jueves,Evento Noche Martes,Evento Noche Miercoles
person,,,,,,,,,,,,,,,,,,,,,,,,,
0008ed71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00091926,86.0,11.0,16.0,73.0,72.0,15.0,40.0,0.0,0.0,0.0,0.0,0.0,...,0.0,50.0,0.0,0.0,0.0,0.0,21.0,16.0,8.0,12.0,28.0,0.0
00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000ba417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.0,0.0,59.0,...,18.0,0.0,88.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000c79fe,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
df_features = df_features.merge(eventos_hora, how='left', on='person')
assert(df_features.shape[0] == df_eventos_up_to_01062018['person'].unique().shape[0])